In [ ]:
# Connecting the Python Code with the google drive
from google.colab import drive

In [ ]:
import pandas as pd

In [ ]:
#load the excel into a dafaframe
df = pd.read_excel('/content/drive/MyDrive/Projects/CoffeeShop.xlsx')
# Display the first few rows to understand its structure
df.head(10)

,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:00,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:00,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:00,It008,1,Taylor,out
3,4,ORD004,2024-02-12 07:18:00,It019,1,Casey,out
4,5,ORD005,2024-02-12 07:23:00,It024,1,Jamie,out
5,6,ORD006,2024-02-12 07:28:00,It001,1,Morgan,in
6,7,ORD006,2024-02-12 07:28:00,It016,1,Morgan,in
7,8,ORD007,2024-02-12 07:33:00,It005,1,Riley,out
8,9,ORD007,2024-02-12 07:33:00,It020,1,Riley,NaN
9,10,ORD008,2024-02-12 07:39:00,It006,1,Cameron,in


Step 1: Data Loading Analysis

Upon loading the data from the ‘CoffeeShop.xlsx’ file, we observed a dataset that includes order details from a coffee shop. The columns indicate the order ID, creation time, item ID, quantity, customer name, and whether the order was for dine-in or takeaway. This initial glimpse into the dataset provided us with the necessary context to plan our analysis, specifically pointing towards understanding customer ordering patterns, potentially leading to inventory management or sales forecasting.

In [ ]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Review data types for each column
data_types = df.dtypes
print("\nData Types:")
print(data_types)

# Handle missing values - Example: Filling missing 'in_or_out' values with 'unknown'
df['in_or_out'].fillna('unknown', inplace=True)

# Extract hour from 'created_at' for potential feature
df['order_hour'] = df['created_at'].dt.hour

# Encode categorical variables - One-hot encoding of 'item_id' and 'in_or_out'
data_encoded = pd.get_dummies(df, columns=['item_id', 'in_or_out'])

# Display the first few rows of the processed data to verify changes
print(data_encoded.head())


Missing Values:
row_id         0
order_id       0
created_at     0
item_id        0
quantity       0
cust_name      0
in_or_out     32
dtype: int64

Data Types:
row_id                 int64
order_id              object
created_at    datetime64[ns]
item_id               object
quantity               int64
cust_name             object
in_or_out             object
dtype: object
   row_id order_id          created_at  quantity cust_name  order_hour  \
0       1   ORD001 2024-02-12 07:04:00         1      Alex           7   
1       2   ORD002 2024-02-12 07:09:00         1    Jordan           7   
2       3   ORD003 2024-02-12 07:14:00         1    Taylor           7   
3       4   ORD004 2024-02-12 07:18:00         1     Casey           7   
4       5   ORD005 2024-02-12 07:23:00         1     Jamie           7   

   item_id_It001  item_id_It0010  item_id_It002  item_id_It003  ...  \
0              0               0              0              0  ...   
1              0               0   

Step 2: Data Exploration and Preprocessing Analysis

The exploration revealed missing values in the ‘in_or_out’ column, indicating some orders lacked information on whether they were for dine-in or takeaway. This missing data could impact analyses related to customer behavior patterns based on dining preferences.

Handling Missing Values: Filling missing ‘in_or_out’ values with 'unknown' allows for retaining these rows in the dataset without introducing bias towards either 'in' or 'out' categories. This approach acknowledges the gap in data while preventing potential distortion in analysis outcomes.

Feature Engineering: Extracting the hour from the ‘created_at’ timestamp as ‘order_hour’ is insightful for understanding peak ordering times. This information could be vital for staffing decisions or marketing promotions targeting specific times of the day.

Categorical Encoding: The one-hot encoding of ‘item_id’ and ‘in_or_out’ prepares the dataset for regression analysis by converting
categorical variables into a numerical format. This step is critical as it transforms the data into a structure that's compatible with regression models, allowing for the exploration of how different items and order types (dine-in vs. takeaway) influence the quantity ordered.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Define features (X) and target variable (y)
X = data_encoded.drop(columns=['row_id', 'order_id', 'created_at', 'cust_name', 'quantity'])
y = data_encoded['quantity']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
lin_reg = LinearRegression()


Step 3: Model Selection Analysis

The choice of a Linear Regression model was based on the dataset's nature and the analysis goals. Linear Regression was deemed suitable for several reasons:

Predictive Analysis: The goal to predict the quantity of items ordered based on factors like order time, item type, and dining preference aligns well with Linear Regression's capabilities. It allows us to quantify the influence of these factors on order quantities.

Interpretability: Given the business context of the coffee shop, understanding the relationship between different variables (e.g., time of day, dine-in vs. takeaway) and order quantities is as important as the prediction itself. Linear Regression provides a straightforward interpretation of these relationships through its coefficients.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Train the model
lin_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lin_reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)


Mean Squared Error: 0.0
R^2 Score: 1.0


Step 4: Training the Model and Evaluation Analysis

The evaluation of the Linear Regression model showed perfect prediction accuracy, which is highly unusual and prompts a deeper examination. In real-world scenarios, such a result could indicate overfitting or data leakage.

Data Leakage Concerns: The perfect scores suggest that the model might have had access to information it wouldn't have in a real-world prediction scenario. This could be due to how the features were encoded or selected.

Overfitting: Although overfitting is more commonly associated with complex models, the unexpected result here suggests the model might be too closely fitted to the training data, raising doubts about its generalizability.

Model Reevaluation: The perfect scores necessitate a reevaluation of the model and possibly the data preparation steps to ensure a realistic assessment of the model's predictive capabilities.